# Dataset Visualization

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/WoodOxen/tactics2d/blob/master/docs/tutorial/dataset_visualization.ipynb)

This notebook records the codes that reproduce the demos in [documentation/dataset-overview](https://tactics2d.readthedocs.io/en/latest/dataset-overview). The first section is executable because it calls the data from the sample dataset folder. The second section needs the users to download the corresponding datasets to display results.

In [ ]:
%%capture
!pip install tactics2d
!tqdm

In [ ]:
import tactics2d

print(tactics2d.__version__)

In [ ]:
%matplotlib notebook

import os
import requests
import warnings

warnings.filterwarnings("ignore")

import json
import xml.etree.ElementTree as ET

from tqdm import tqdm
import numpy as np
import matplotlib as mpl

from tactics2d.dataset_parser import (
    ArgoverseParser,
    DLPParser,
    InteractionParser,
    LevelXParser,
    NuPlanParser,
    WOMDParser,
)
from tactics2d.map.parser import OSMParser
from tactics2d.traffic.scenario_display import ScenarioDisplay

In [ ]:
mpl.rcParams.update(
    {
        "figure.dpi": 300,
        "figure.frameon": False,
        "font.family": "Dejavu Serif",
        "font.size": 10,
        "font.stretch": "semi-expanded",
        "animation.html": "jshtml",
        "animation.embed_limit": 5000,
    }
)

In [ ]:
def download_file(url, filename):
    if not os.path.exists(filename):
        r = requests.get(url)
        if r.status_code == 200:
            total_size = int(r.headers.get("content-length", 0))
            block_size = 1024

        progress_bar = tqdm(total=total_size, unit="iB", unit_scale=True)
        with open(filename, "wb") as f:
            for data in r.iter_content(block_size):
                progress_bar.update(len(data))
                f.write(data)
        progress_bar.close()

In [ ]:
config_path = "./map.config"

download_file(
    "https://raw.githubusercontent.com/WoodOxen/tactics2d/master/tactics2d/dataset_parser/map.config",
    config_path,
)

with open(config_path, "r") as f:
    configs = json.load(f)

scenario_display = ScenarioDisplay()
lanelet2_parser = OSMParser(lanelet2=True)

## 1. Sample scenario demostration

### Argoverse 2

In [ ]:
download_file(
    "https://raw.githubusercontent.com/SCP-CN-001/trajectory_dataset_support/main/trajectory_sample/Argoverse/train/0a0a2bb7-c4f4-44cd-958a-9ee15cb34aca/log_map_archive_0a0a2bb7-c4f4-44cd-958a-9ee15cb34aca.json",
    "log_map_archive_0a0a2bb7-c4f4-44cd-958a-9ee15cb34aca.json",
)

download_file(
    "https://raw.githubusercontent.com/SCP-CN-001/trajectory_dataset_support/main/trajectory_sample/Argoverse/train/0a0a2bb7-c4f4-44cd-958a-9ee15cb34aca/scenario_0a0a2bb7-c4f4-44cd-958a-9ee15cb34aca.parquet",
    "scenario_0a0a2bb7-c4f4-44cd-958a-9ee15cb34aca.parquet",
)


In [ ]:
def demo_argoverse(
    file, map_file, folder, xlim=None, ylim=None, fps=10, interval=100, export_to=None
):
    dataset_parser = ArgoverseParser()
    trajectories, actual_time_range = dataset_parser.parse_trajectory(file, folder)

    frames = np.arange(actual_time_range[0], actual_time_range[1], interval)

    map_ = dataset_parser.parse_map(map_file, folder)

    scenario_display = ScenarioDisplay()
    scenario_display.reset()
    if xlim is None or ylim is None:
        ax_settings = {"aspect": "equal"}
        fig_size = (4, 3)
    else:
        ax_settings = {"aspect": "equal", "xlim": xlim, "ylim": ylim}
        fig_size = (4, round((ylim[1] - ylim[0]) / (xlim[1] - xlim[0]) * 4, 1))

    animation = scenario_display.display(
        trajectories, map_, interval, frames, fig_size, **ax_settings
    )

    if export_to is None:
        return animation
    else:
        animation.save(filename=export_to, writer="ffmpeg", fps=fps, dpi=300)

In [ ]:
# plot Argoverse 2 sample scenario
demo_argoverse(
    "scenario_0a0a2bb7-c4f4-44cd-958a-9ee15cb34aca.parquet",
    "log_map_archive_0a0a2bb7-c4f4-44cd-958a-9ee15cb34aca.json",
    "./",
    xlim=[1800, 2100],
    ylim=[500, 750],
)

### DLP

In [ ]:
download_file(
    "https://datadryad.org/stash/downloads/file_stream/2654062",
    "DJI_0012_agents.json"
)
download_file(
    "https://datadryad.org/stash/downloads/file_stream/2654066",
    "DJI_0012_frames.json"
)
download_file(
    "https://datadryad.org/stash/downloads/file_stream/2654081",
    "DJI_0012_instances.json"
)
download_file(
    "https://datadryad.org/stash/downloads/file_stream/2654064",
    "DJI_0012_obstacles.json"
)
download_file(
    "https://raw.githubusercontent.com/SCP-CN-001/trajectory_dataset_support/main/map/DLP/DLP.osm",
    "DLP.osm"
)

In [ ]:
# plot DLP sample scenario
dlp_time_range = (0, 20000)
dlp_fps = 25
dlp_interval = int(1000 / dlp_fps)

dataset_parser = DLPParser()
trajectories, actual_time_range = dataset_parser.parse_trajectory(
    "DJI_0012_agents.json", "./", dlp_time_range
)

frames = np.arange(actual_time_range[0], actual_time_range[1], dlp_interval)

map_config = configs["DLP"]
map_path = "./DLP.osm"
map_root = ET.parse(map_path).getroot()
map_ = lanelet2_parser.parse(map_root, map_config["project_rule"], map_config["gps_origin"], map_config)

scenario_display.reset()
ax_settings = {"aspect": "equal", "xlim": [-5, 145], "ylim": [0, 80]}
scenario_display.display(trajectories, map_, dlp_interval, frames, (5, 2.7))
# animation.save(filename="./samples/imgs/dlp_sample.gif", writer="ffmpeg", fps=dlp_fps, dpi=300)

### INTERACTION

In [ ]:
download_file(
    "https://raw.githubusercontent.com/SCP-CN-001/trajectory_dataset_support/main/map/INTERACTION/DR_USA_Intersection_EP0.osm",
    "DR_USA_Intersection_EP0.osm"
)
download_file(
    "https://raw.githubusercontent.com/SCP-CN-001/trajectory_dataset_support/main/trajectory_sample/INTERACTION/recorded_trackfiles/DR_USA_Intersection_EP0/pedestrian_tracks_000.csv",
    "pedestrian_tracks_000.csv"
)
download_file(
    "https://raw.githubusercontent.com/SCP-CN-001/trajectory_dataset_support/main/trajectory_sample/INTERACTION/recorded_trackfiles/DR_USA_Intersection_EP0/vehicle_tracks_000.csv",
    "vehicle_tracks_000.csv"
)

In [ ]:
def demo_interaction(
    file,
    folder,
    map_name,
    xlim=None,
    ylim=None,
    time_range=(0, 10000),
    fps=10,
    interval=100,
    export_to=None,
):
    dataset_parser = InteractionParser()
    trajectories, actual_time_range = dataset_parser.parse_trajectory(file, folder, time_range)

    frames = np.arange(actual_time_range[0], actual_time_range[1], interval)

    map_config = configs[map_name]
    map_path = map_config["osm_path"].split("/")[-1]
    map_root = ET.parse(map_path).getroot()
    map_ = lanelet2_parser.parse(map_root, map_config["project_rule"], map_config["gps_origin"], map_config)

    scenario_display = ScenarioDisplay()
    scenario_display.reset()
    if xlim is None or ylim is None:
        ax_settings = {"aspect": "equal"}
        fig_size = (4, 3)
    else:
        ax_settings = {"aspect": "equal", "xlim": xlim, "ylim": ylim}
        fig_size = (4, round((ylim[1] - ylim[0]) / (xlim[1] - xlim[0]) * 4, 1))

    animation = scenario_display.display(
        trajectories, map_, interval, frames, fig_size, **ax_settings
    )

    if export_to is None:
        return animation
    else:
        animation.save(filename=export_to, writer="ffmpeg", fps=fps, dpi=300)

In [ ]:
# plot INTERACTION sample scenario
demo_interaction(
    "vehicle_tracks_000.csv",
    "./",
    "DR_USA_Intersection_EP0",
    xlim=[945, 1050],
    ylim=[965, 1025],
)

### WOMD

In [ ]:
# plot WOMD sample scenario
womd_fps = 20
womd_interval = int(1000 / womd_fps)

dataset_parser = WOMDParser()
trajectories, actual_time_range = dataset_parser.parse_trajectory(
    file="motion_data_one_scenario.tfrecord", folder="./tactics2d/data/trajectory_sample/WOMD"
)

map_ = dataset_parser.parse_map(
    file="motion_data_one_scenario.tfrecord", folder="./tactics2d/data/trajectory_sample/WOMD"
)

scenario_display.reset()
ax_settings = {"aspect": "equal", "xlim": [-7890, -7710], "ylim": [-6775, -6600]}
animation = scenario_display.display(
    trajectories, map_, womd_interval, actual_time_range, (5, 5.5), **ax_settings
)
animation
animation.save(filename="./samples/imgs/womd_sample.gif", writer="ffmpeg", fps=womd_fps, dpi=300)

## Plot of the datasets

This section provides the full code to plot all demos in the documentation. To run this section, the users need to download the corresponding datasets and put them in the `data` folder.

## LevelX

According to the request from the official data provider, the data is not allowed to be shared. The users can download the data from the official website and put it in the `data` folder. The data is available at [HighD website](https://levelxdata.com/highd-dataset/).

In [ ]:
def demo_levelx(
    dataset,
    file,
    folder,
    xlim=None,
    ylim=None,
    time_range=(0, 10000),
    fps=25,
    interval=40,
    export_to=None,
):
    dataset_parser = LevelXParser(dataset)
    trajectories, actual_time_range = dataset_parser.parse_trajectory(file, folder, time_range)

    frames = np.arange(actual_time_range[0], actual_time_range[1], interval)

    location = dataset_parser.get_location(file, folder)
    map_name = f"{dataset}_{location}"

    map_config = configs[map_name]
    map_path = os.path.join(map_folder, map_config["osm_path"])
    map_root = ET.parse(map_path).getroot()
    lanelet2_parser = OSMParser(lanelet2=True)
    map_ = lanelet2_parser.parse(
        map_root, map_config["project_rule"], map_config["gps_origin"], map_config
    )

    scenario_display = ScenarioDisplay()
    scenario_display.reset()
    if xlim is None or ylim is None:
        ax_settings = {"aspect": "equal"}
        fig_size = (4, 3)
    else:
        ax_settings = {"aspect": "equal", "xlim": xlim, "ylim": ylim}
        fig_size = (4, round((ylim[1] - ylim[0]) / (xlim[1] - xlim[0]) * 4, 1))

    animation = scenario_display.display(
        trajectories, map_, interval, frames, fig_size, **ax_settings
    )

    if export_to is None:
        return animation
    else:
        animation.save(filename=export_to, writer="ffmpeg", fps=fps, dpi=300)

### HighD

Plot sample data from the HighD dataset.

In [ ]:
demo_levelx(
    "highD",
    "01_tracks.csv",
    "./tactics2d/data/trajectory_sample/highD/data",
    xlim=[0, 300],
    ylim=[-25, 5],
)

In [ ]:
demo_levelx(
    "highD",
    "11_tracks.csv",
    "./tactics2d/data/trajectory/highD/data",
    xlim=[0, 300],
    ylim=[-35, 5],
    export_to="./samples/imgs/highD_loc_1.gif",
)

In [ ]:
demo_levelx(
    "highD",
    "01_tracks.csv",
    "./tactics2d/data/trajectory/highD/data",
    xlim=[0, 300],
    ylim=[-25, 5],
    export_to="./samples/imgs/highD_loc_2.gif",
)

In [ ]:
demo_levelx(
    "highD",
    "04_tracks.csv",
    "./tactics2d/data/trajectory/highD/data",
    xlim=[0, 300],
    ylim=[-30, 5],
    export_to="./samples/imgs/highD_loc_3.gif",
)

In [ ]:
demo_levelx(
    "highD",
    "07_tracks.csv",
    "./tactics2d/data/trajectory/highD/data",
    xlim=[0, 300],
    ylim=[-30, 5],
    export_to="./samples/imgs/highD_loc_4.gif",
)

In [ ]:
demo_levelx(
    "highD",
    "15_tracks.csv",
    "./tactics2d/data/trajectory/highD/data",
    xlim=[0, 300],
    ylim=[-25, 5],
    time_range=(10000, 20000),
    export_to="./samples/imgs/highD_loc_5.gif",
)

In [ ]:
demo_levelx(
    "highD",
    "58_tracks.csv",
    "./tactics2d/data/trajectory/highD/data",
    xlim=[0, 300],
    ylim=[-30, 10],
    time_range=(10000, 20000),
    export_to="./samples/imgs/highD_loc_6.gif",
)

### InD

Plot sample data from the InD dataset.

In [ ]:
demo_levelx(
    "inD",
    "00_tracks.csv",
    "./tactics2d/data/trajectory_sample/inD/data",
    xlim=[50, 200],
    ylim=[-125, -25],
)

In [ ]:
demo_levelx(
    "inD",
    "07_tracks.csv",
    "./tactics2d/data/trajectory/inD/data",
    xlim=[0, 120],
    ylim=[-75, 20],
    export_to="./samples/imgs/inD_loc_1.gif",
)

In [ ]:
demo_levelx(
    "inD",
    "18_tracks.csv",
    "./tactics2d/data/trajectory/inD/data",
    xlim=[0, 150],
    ylim=[-60, 0],
    export_to="./samples/imgs/inD_loc_2.gif",
)

In [ ]:
demo_levelx(
    "inD",
    "30_tracks.csv",
    "./tactics2d/data/trajectory/inD/data",
    xlim=[-25, 100],
    ylim=[-75, 0],
    export_to="./samples/imgs/inD_loc_3.gif",
)

In [ ]:
demo_levelx(
    "inD",
    "00_tracks.csv",
    "./tactics2d/data/trajectory/inD/data",
    xlim=[50, 240],
    ylim=[-125, 0],
    export_to="./samples/imgs/inD_loc_4.gif",
)

### RounD

Plot sample data from the RounD dataset.

In [ ]:
demo_levelx(
    "rounD",
    "00_tracks.csv",
    "./tactics2d/data/trajectory_sample/rounD/data",
    xlim=[50, 200],
    ylim=[-145, -25],
)

In [ ]:
demo_levelx(
    "rounD",
    "02_tracks.csv",
    "./tactics2d/data/trajectory/rounD/data",
    xlim=[0, 150],
    ylim=[-100, 5],
    export_to="./samples/imgs/rounD_loc_0.gif",
)

In [ ]:
demo_levelx(
    "rounD",
    "00_tracks.csv",
    "./tactics2d/data/trajectory/rounD/data",
    xlim=[50, 200],
    ylim=[-150, -25],
    export_to="./samples/imgs/rounD_loc_1.gif",
)

In [ ]:
demo_levelx(
    "rounD",
    "01_tracks.csv",
    "./tactics2d/data/trajectory/rounD/data",
    xlim=[100, 175],
    ylim=[-125, -25],
    export_to="./samples/imgs/rounD_loc_2.gif",
)

### ExiD

Plot sample data from the ExiD dataset.

In [ ]:
demo_levelx(
    "exiD",
    "00_tracks.csv",
    "./tactics2d/data/trajectory_sample/rounD/data",
    xlim=[300, 600],
    ylim=[-500, -100],
)

In [ ]:
demo_levelx(
    "exiD",
    "00_tracks.csv",
    "./tactics2d/data/trajectory/exiD/data",
    xlim=[300, 600],
    ylim=[-500, -100],
    export_to="./samples/imgs/exiD_loc_0.gif",
)

In [ ]:
demo_levelx(
    "exiD",
    "19_tracks.csv",
    "./tactics2d/data/trajectory/exiD/data",
    xlim=[310, 600],
    ylim=[-390, -90],
    export_to="./samples/imgs/exiD_loc_1.gif",
)

In [ ]:
demo_levelx(
    "exiD",
    "39_tracks.csv",
    "./tactics2d/data/trajectory/exiD/data",
    xlim=[550, 880],
    ylim=[-600, -200],
    export_to="./samples/imgs/exiD_loc_2.gif",
)

In [ ]:
demo_levelx(
    "exiD",
    "53_tracks.csv",
    "./tactics2d/data/trajectory/exiD/data",
    xlim=[150, 550],
    ylim=[-340, -50],
    export_to="./samples/imgs/exiD_loc_3.gif",
)

In [ ]:
demo_levelx(
    "exiD",
    "61_tracks.csv",
    "./tactics2d/data/trajectory/exiD/data",
    xlim=[200, 600],
    ylim=[-350, -50],
    export_to="./samples/imgs/exiD_loc_4.gif",
)

In [ ]:
demo_levelx(
    "exiD",
    "73_tracks.csv",
    "./tactics2d/data/trajectory/exiD/data",
    xlim=[150, 650],
    ylim=[-300, -140],
    export_to="./samples/imgs/exiD_loc_5.gif",
)

In [ ]:
demo_levelx(
    "exiD",
    "78_tracks.csv",
    "./tactics2d/data/trajectory/exiD/data",
    xlim=[150, 600],
    ylim=[-240, -130],
    export_to="./samples/imgs/exiD_loc_6.gif",
)

### Argoverse

In [ ]:
demo_argoverse(
    "scenario_0a51f8f5-3e8a-4d40-87e6-cb325c9c44ce.parquet",
    "log_map_archive_0a51f8f5-3e8a-4d40-87e6-cb325c9c44ce.json",
    "./tactics2d/data/trajectory/Argoverse/test/0a51f8f5-3e8a-4d40-87e6-cb325c9c44ce",
    xlim=[3400, 3700],
    ylim=[-2000, -1800],
    export_to="./samples/imgs/argoverse_sample.gif",
)

### INTERACTION

In [ ]:
demo_interaction(
    "vehicle_tracks_000.csv",
    "./tactics2d/data/trajectory/INTERACTION/recorded_trackfiles/DR_CHN_Merging_ZS",
    "DR_CHN_Merging_ZS",
    xlim=[1000, 1140],
    ylim=[925, 975],
    export_to="./samples/imgs/DR_CHN_Merging_ZS.gif",
)

In [ ]:
demo_interaction(
    "vehicle_tracks_000.csv",
    "./tactics2d/data/trajectory/INTERACTION/recorded_trackfiles/DR_CHN_Roundabout_LN",
    "DR_CHN_Roundabout_LN",
    xlim=[920, 1070],
    ylim=[960, 1050],
    export_to="./samples/imgs/DR_CHN_Roundabout_LN.gif",
)

In [ ]:
demo_interaction(
    "vehicle_tracks_000.csv",
    "./tactics2d/data/trajectory/INTERACTION/recorded_trackfiles/DR_DEU_Merging_MT",
    "DR_DEU_Merging_MT",
    xlim=[890, 1005],
    ylim=[1000, 1015],
    export_to="./samples/imgs/DR_DEU_Merging_MT.gif",
)

In [ ]:
demo_interaction(
    "vehicle_tracks_000.csv",
    "./tactics2d/data/trajectory/INTERACTION/recorded_trackfiles/DR_DEU_Roundabout_OF",
    "DR_DEU_Roundabout_OF",
    xlim=[950, 1050],
    ylim=[960, 1030],
    export_to="./samples/imgs/DR_DEU_Roundabout_OF.gif",
)

In [ ]:
demo_interaction(
    "vehicle_tracks_000.csv",
    "./tactics2d/data/trajectory/INTERACTION/recorded_trackfiles/DR_USA_Intersection_EP0",
    "DR_USA_Intersection_EP0",
    xlim=[950, 1050],
    ylim=[965, 1020],
    export_to="./samples/imgs/DR_USA_Intersection_EP0.gif",
)

In [ ]:
demo_interaction(
    "vehicle_tracks_000.csv",
    "./tactics2d/data/trajectory/INTERACTION/recorded_trackfiles/DR_USA_Intersection_EP1",
    "DR_USA_Intersection_EP1",
    xlim=[950, 1095],
    ylim=[950, 1020],
    export_to="./samples/imgs/DR_USA_Intersection_EP1.gif",
)

In [ ]:
demo_interaction(
    "vehicle_tracks_000.csv",
    "./tactics2d/data/trajectory/INTERACTION/recorded_trackfiles/DR_USA_Intersection_GL",
    "DR_USA_Intersection_GL",
    xlim=[940, 1040],
    ylim=[940, 1020],
    export_to="./samples/imgs/DR_USA_Intersection_GL.gif",
)

In [ ]:
demo_interaction(
    "vehicle_tracks_000.csv",
    "./tactics2d/data/trajectory/INTERACTION/recorded_trackfiles/DR_USA_Intersection_MA",
    "DR_USA_Intersection_MA",
    xlim=[970, 1050],
    ylim=[960, 1040],
    export_to="./samples/imgs/DR_USA_Intersection_MA.gif",
)

In [ ]:
demo_interaction(
    "vehicle_tracks_000.csv",
    "./tactics2d/data/trajectory/INTERACTION/recorded_trackfiles/DR_USA_Roundabout_EP",
    "DR_USA_Roundabout_EP",
    xlim=[950, 1095],
    ylim=[980, 1050],
    export_to="./samples/imgs/DR_USA_Roundabout_EP.gif",
)

In [ ]:
demo_interaction(
    "vehicle_tracks_000.csv",
    "./tactics2d/data/trajectory/INTERACTION/recorded_trackfiles/DR_USA_Roundabout_FT",
    "DR_USA_Roundabout_FT",
    xlim=[960, 1060],
    ylim=[970, 1025],
    export_to="./samples/imgs/DR_USA_Roundabout_FT.gif",
)

In [ ]:
demo_interaction(
    "vehicle_tracks_001.csv",
    "./tactics2d/data/trajectory/INTERACTION/recorded_trackfiles/DR_USA_Roundabout_SR",
    "DR_USA_Roundabout_SR",
    xlim=[910, 1050],
    ylim=[980, 1060],
    export_to="./samples/imgs/DR_USA_Roundabout_SR.gif",
)

In [ ]:
demo_interaction(
    "vehicle_tracks_001.csv",
    "./tactics2d/data/trajectory/INTERACTION/recorded_trackfiles/TC_BGR_Intersection_VA",
    "TC_BGR_Intersection_VA",
    xlim=[965, 1025],
    ylim=[975, 1035],
    export_to="./samples/imgs/DR_BGR_Intersection_VA.gif",
)

### NuPlan

Because the map of NuPlan is extremely large, we suggest you to run the following codes in the local environment.

In [ ]:
# plot NuPlan sample scenario
nuplan_fps = 20
nuplan_interval = int(1000 / nuplan_fps)

dataset_parser = NuPlanParser()
participants, actual_time_range = dataset_parser.parse_trajectory(
    "2021.08.26.18.24.36_veh-28_00578_00663.db",
    "./tactics2d/data/trajectory/NuPlan/data/cache/train_boston",
)

frames = [t for t in actual_time_range if t < actual_time_range[0] + 10000]
location = dataset_parser.get_location(
    "2021.08.26.18.24.36_veh-28_00578_00663.db",
    "./tactics2d/data/trajectory_sample/NuPlan/data/cache/train_boston",
)
map_ = dataset_parser.parse_map(configs[location]["gpkg_path"], map_folder)

scenario_display.reset()
ax_settings = {"aspect": "equal", "xlim": [331420, 331550], "ylim": [4689360, 4689470]}
animation = scenario_display.display(
    participants, map_, nuplan_interval, frames, (5, 4.2), **ax_settings
)
animation.save(
    filename="./samples/imgs/nuplan_boston.gif", writer="ffmpeg", fps=nuplan_fps, dpi=300
)

In [ ]:
dataset_parser = NuPlanParser()
participants, actual_time_range = dataset_parser.parse_trajectory(
    "2021.08.17.13.15.12_veh-45_00324_00489.db",
    "./tactics2d/data/trajectory/NuPlan/data/cache/train_pittsburgh/",
)

frames = [t for t in actual_time_range if t < actual_time_range[0] + 10000]
location = dataset_parser.get_location(
    "2021.08.17.13.15.12_veh-45_00324_00489.db",
    "./tactics2d/data/trajectory/NuPlan/data/cache/train_pittsburgh/",
)
map_ = dataset_parser.parse_map(configs[location]["gpkg_path"], map_folder)

x_min = np.inf
x_max = -np.inf
y_min = np.inf
y_max = -np.inf

for participant in participants.values():
    for state in participant.trajectory.history_states.values():
        x, y = state.location
        x_min = min(x_min, x)
        x_max = max(x_max, x)
        y_min = min(y_min, y)
        y_max = max(y_max, y)

print(x_min, x_max, y_min, y_max)

scenario_display.reset()
ax_settings = {"aspect": "equal", "xlim": [x_min, x_max], "ylim": [y_min, y_max]}
scenario_display.display(participants, map_, nuplan_interval, frames, (5, 4.2), **ax_settings)
# animation = scenario_display.display(participants, map_, nuplan_interval, frames, (5, 4.2), **ax_settings)
# animation.save(filename="./samples/imgs/nuplan_pittsburghgif", writer="ffmpeg", fps=nuplan_fps, dpi=300)

In [ ]:
dataset_parser = NuPlanParser()
participants, actual_time_range = dataset_parser.parse_trajectory(
    "2021.08.18.06.04.33_veh-51_00183_00300.db",
    "./tactics2d/data/trajectory/NuPlan/data/cache/train_singapore/",
)

frames = [t for t in actual_time_range if t < actual_time_range[0] + 10000]
location = dataset_parser.get_location(
    "2021.08.18.06.04.33_veh-51_00183_00300.db",
    "./tactics2d/data/trajectory/NuPlan/data/cache/train_singapore/",
)
map_ = dataset_parser.parse_map(configs[location]["gpkg_path"], map_folder)

x_min = np.inf
x_max = -np.inf
y_min = np.inf
y_max = -np.inf

for participant in participants.values():
    for state in participant.trajectory.history_states.values():
        x, y = state.location
        x_min = min(x_min, x)
        x_max = max(x_max, x)
        y_min = min(y_min, y)
        y_max = max(y_max, y)

print(x_min, x_max, y_min, y_max)

scenario_display.reset()
ax_settings = {"aspect": "equal", "xlim": [365021, 365314], "ylim": [142988, 143251]}
scenario_display.display(participants, map_, nuplan_interval, frames, (5, 4.2), **ax_settings)
# animation = scenario_display.display(participants, map_, nuplan_interval, frames, (5, 4.2), **ax_settings)
# animation.save(filename="./samples/imgs/nuplan_pittsburghgif", writer="ffmpeg", fps=nuplan_fps, dpi=300)